In [24]:
import pandas as pd

df = pd.read_table('spam.csv',
                    sep=',', 
                #    header=None,
                   encoding="ISO-8859-1",
                #    names=['label', 'message']
                   )
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [25]:
df = df.rename({'v1': 'label', 'v2': 'message'}, axis=1)  # new method
df.loc[:, df.select_dtypes('O').columns] = df.select_dtypes('O').astype(str)
df.dtypes

label         object
message       object
Unnamed: 2    object
Unnamed: 3    object
Unnamed: 4    object
dtype: object

In [26]:
df['label'] = df.label.map({'ham': 0, 'spam': 1})
df

,label,message,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,0,"Go until jurong point, crazy.. Available only ...",nan,nan,nan
1,0,Ok lar... Joking wif u oni...,nan,nan,nan
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,nan,nan,nan
3,0,U dun say so early hor... U c already then say...,nan,nan,nan
4,0,"Nah I don't think he goes to usf, he lives aro...",nan,nan,nan
...,...,...,...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...,nan,nan,nan
5568,0,Will Ì_ b going to esplanade fr home?,nan,nan,nan
5569,0,"Pity, * was in mood for that. So...any other s...",nan,nan,nan
5570,0,The guy did some bitching but I acted like i'd...,nan,nan,nan


In [27]:
# преобразовать все символы в сообщении в нижний регистр:
df['message'] = df.message.str.lower()
# убрать все знаки препинания:
df['message'] = df.message.str.replace('[^\w\s]', '')
df

C:\Users\Anton\AppData\Local\Temp\ipykernel_17668\718531557.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['message'] = df.message.str.replace('[^\w\s]', '')


,label,message,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,0,go until jurong point crazy available only in ...,nan,nan,nan
1,0,ok lar joking wif u oni,nan,nan,nan
2,1,free entry in 2 a wkly comp to win fa cup fina...,nan,nan,nan
3,0,u dun say so early hor u c already then say,nan,nan,nan
4,0,nah i dont think he goes to usf he lives aroun...,nan,nan,nan
...,...,...,...,...,...
5567,1,this is the 2nd time we have tried 2 contact u...,nan,nan,nan
5568,0,will ì_ b going to esplanade fr home,nan,nan,nan
5569,0,pity was in mood for that soany other suggest...,nan,nan,nan
5570,0,the guy did some bitching but i acted like id ...,nan,nan,nan


In [28]:
# разделить сообщения на отдельные слова
# import nltk
# nltk.download()


In [29]:
df['message'] = df['message'].apply(nltk.word_tokenize)

Идея стемминга состоит в том, чтобы нормализовать наш текст, чтобы все варианты слов имели одно и то же значение, независимо от времени. Одним из самых популярных алгоритмов стемминга является Стеммер Портера:

In [31]:
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()
 
df['message'] = df['message'].apply(lambda x: [stemmer.stem(y) for y in x])

Наконец, мы преобразуем данные в события, которые будут функциями, которые мы будем использовать в нашей модели:

In [32]:
from sklearn.feature_extraction.text import CountVectorizer

# This converts the list of words into space-separated strings
df['message'] = df['message'].apply(lambda x: ' '.join(x))

count_vect = CountVectorizer()
counts = count_vect.fit_transform(df['message'])

Мы могли бы оставить его как простое количество слов в сообщении, но лучше использовать частоту термина, обратную частоте документа, более известную как tf-idf:

In [33]:
from sklearn.feature_extraction.text import TfidfTransformer

transformer = TfidfTransformer().fit(counts)

counts = transformer.transform(counts)

## Обучение модели

In [34]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(counts, df['label'], test_size=0.1, random_state=69)

Для задач классификации текста хорошо подходит полиномиальный наивный байесовский классификатор:

In [36]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB

model = GaussianNB().fit(X_train.toarray(), y_train)

## Оценка модели

In [38]:
import numpy as np

predicted = model.predict(X_test.toarray())

print(np.mean(predicted == y_test))

0.8853046594982079


In [39]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, predicted))

[[426  54]
 [ 10  68]]
